In [45]:
import glob, os
import pandas as pd
import uuid

In [330]:
'''
Load all xml files parsed with AH-1954-1973_XML_parser.py
'''

allFiles = glob.glob('../../../Dropbox/datasets/Allerhande_1954-1973/allerhande_processed/**/*.csv', recursive=True)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=None)
    df['filename'] = os.path.basename(file_)
    list_.append(df)
data = pd.concat(list_)

In [331]:
data['year'] = data['filename'].str[:4]
data['month'] = data['filename'].str[5:7]
data['page'] = data['filename'].str[8:-4]
data.columns.values[0] = 'text'
data['text'] = data['text'].astype(str)
data = data.drop_duplicates('text', keep='first')
data['text_clean'] = data['text'].str.replace('¬\n', '')
data['text_clean'] = data['text_clean'].str.replace('\n', ' ')
#remove short articles less than 10 characters / words
data = data.sort_values(by=['year', 'month'])
data['text_clean'] = data['text_clean'].str.strip()
data = data.drop('text', axis=1)
data = data.reset_index()

In [332]:
#get id for text (second part) that start with a lowercase character this is due to an incorrect separation or ocr
#and id for text part before that part, probably the first part of the split article (first part)

first_part = data[data['text_clean'].str.split().str.get(0).apply(lambda x: x.islower())].index - 1
second_part = data[data['text_clean'].str.split().str.get(0).apply(lambda x: x.islower())].index

#combine these two text parts
text = []
for index, row in data.iterrows():
    if index in second_part:
        #index_2 = index - 1
        text.append(data.iloc[index -1]['text_clean'] + data.iloc[index]['text_clean'])
        
# #replace existing text with joined articles. 
temp = data.loc[first_part] 
temp = temp.drop('text_clean', axis=1)
temp['text_clean'] = text

#remove split articles from data
data = data[~data.index.isin(first_part)]
data = data[~data.index.isin(second_part)]

data = pd.concat([data, temp])
data = data.reset_index()

In [333]:
#repeat same tricks for articles with an enter
first_part2 = data[data['text_clean'].str.endswith('¬')].index
second_part2 = data[data['text_clean'].str.endswith('¬')].index + 1

text2 = []
for index, row in data.iterrows():
    if index in first_part2:
        text2.append(data.iloc[index]['text_clean'] + data.iloc[index + 1]['text_clean'])
        
temp_2 = data.loc[first_part2]
temp_2 = temp_2.drop('text_clean', axis=1)
temp_2['text_clean'] = text2

#remove articles split with enter
data = data[~data.index.isin(first_part2)]
data = data[~data.index.isin(second_part2)]

data = pd.concat([data, temp_2])
data['text_clean'] = data['text_clean'].str.replace('¬', '')

col_drop = ['level_0', 'index', 'filename']
data.drop(col_drop, axis=1, inplace=True)

In [334]:
# remove articles shorter than 5 words
data['length'] = data['text_clean'].str.split().apply(lambda x: len(x))
data = data[data['length'] >= 5]

In [337]:
data.sort_values(['year', 'month', 'page'])

,year,month,page,text_clean,length,uuid
13,1954,12,1,IHIIIIII lil IIIIII il IIIIII SAH001000001 All...,9,07d64564
15,1954,12,1,Honderdduizenden vriendschapsbewijzen over Ned...,5,07d645a0
16,1954,12,1,"Majesteit „Het Banket"" heeft ontbijtkoek en bi...",135,07d645d2
18,1954,12,1,"TE AMANDELSPUS 'URE ROOMBOTER wonder, dat er m...",148,07d6460e
19,1954,12,1,EEN UUR VOOR 15 SECONDEN,5,07d6464a
38391,1954,12,1,VAARDIGE HANDEN pakken de kilometerslange reek...,25,07f1e558
38392,1954,12,1,ijne Majesteit „Het Banket' kort en feestelijk...,7,07f1e594
38393,1954,12,1,Pas wanneer die fabrikant er een briefje bij g...,111,07f1e5d0
38394,1954,12,1,In deze krant treft U een reeks advertenties a...,255,07f1e60c
38395,1954,12,1,Dynamisch bedrijf gaat door met openen van Z.B...,197,07f1e648


In [338]:
data.to_csv('allerhande_1954-1973.csv', index=False)